# Basics #
## 3. Numerical Greeks calculation ##

### Setup ###

In [1]:
import QuantLib as ql

In [2]:
today = ql.Date(8, ql.October, 2014)
ql.Settings.instance().evaluationDate = today

### A somewhat exotic option ###

In [3]:
option = ql.BarrierOption(ql.Barrier.UpIn,
                          120.0, # barrier
                          0.0, # rebate
                          ql.PlainVanillaPayoff(ql.Option.Call, 100.0),
                          ql.EuropeanExercise(ql.Date(8, ql.January, 2015)))

In [4]:
u = ql.SimpleQuote(100.0)
r = ql.SimpleQuote(0.01)
sigma = ql.SimpleQuote(0.20)

In [5]:
riskFreeCurve = ql.FlatForward(0, ql.TARGET(), ql.QuoteHandle(r),ql.Actual360())
volatility = ql.BlackConstantVol(0, ql.TARGET(), ql.QuoteHandle(sigma), ql.Actual360())

In [6]:
process = ql.BlackScholesProcess(ql.QuoteHandle(u),
                                 ql.YieldTermStructureHandle(riskFreeCurve),
                                 ql.BlackVolTermStructureHandle(volatility))

In [7]:
option.setPricingEngine(ql.AnalyticBarrierEngine(process))

In [8]:
print(option.NPV())

1.3657980739109867


In [9]:
print(option.delta())

RuntimeError: delta not provided

In [10]:
u0 = u.value()
h = 0.01

In [11]:
P0 = option.NPV()
print(P0)

1.3657980739109867


In [12]:
u.setValue(u0+h)
P_plus = option.NPV()
print(P_plus)

1.3688112201958083


In [13]:
u.setValue(u0-h)
P_minus = option.NPV()
print(P_minus)

1.3627900998610207


In [14]:
u.setValue(u0)

In [15]:
Delta = (P_plus - P_minus) / (2 * h)
Gamma = (P_plus - 2 * P0 + P_minus) / (h * h)
print(Delta)
print(Gamma)

0.3010560167393761
0.05172234855521651


In [16]:
r0 = r.value()
h = 0.0001
r.setValue(r0 + h)
P_plus = option.NPV()
r.setValue(r0)
Rho = (P_plus - P0) / h
print(Rho)

6.531038494277386


In [17]:
sigma0 = sigma.value()
h = 0.0001
sigma.setValue(sigma0 + h)
P_plus = option.NPV()
sigma.setValue(sigma0)
Vega = (P_plus - P0) / h
print(Vega)

26.52519924198904


In [18]:
ql.Settings.instance().evaluationDate = today + 1
P1 = option.NPV()
h = 1 / 365
Theta = (P1 - P0) / h
print(Theta)

-10.770888399441302
